# Naive Bayes Classifier

In [1]:
# Tập dữ liệu ví dụ

train_x = [
           'just plain boring',
           'entirely predictable and lacks energy',
           'no surprises and very few laughs',
           'very powerful',
           'the most fun film of the summer'
]
train_y = [0, 0, 0, 1, 1]

**Tiền xử lý dữ liệu cơ bản**

In [2]:
def basic_preprocess(text):
    """ Tiền xử lý và tách các câu
    Args:
        text: câu đầu vào. 
        VD: "Tôi đi học"
    Output:
        text_clean: danh sách các từ (token) sau khi chuyển sang chữ thường và
            được phân tách bởi khoảng trắng
    """
    text_clean = text.lower()
    return text_clean.split()

basic_preprocess(train_x[0])

['just', 'plain', 'boring']

**Xây dựng bộ từ điển**

In [3]:
def count_freq_words(corpus, labels):
    """ Xây dựng bộ từ điển tần suất xuất hiện của các từ
    Args:
        corpus: tập danh sách các câu
        labels: tập nhãn tương ứng với các câu trong corpus (0 hoặc 1)
    Output:
        model: bộ từ điển ánh xạ mỗi từ và tần suất xuất hiện của từ đó trong corpus
            key: (word, label)
            value: frequency
            VD: {('boring', 0): 2} => từ boring xuất hiện 2 lần trong các sample thuộc class 0
    """
    model = {}
    for label, sentence in zip(labels, corpus):
        for word in basic_preprocess(sentence):
            # Định nghĩa key của model là tuple (word, label)
            pair = (word, label)
            # Nếu key đã tồn tại trong model thì tăng value lên 1
            if pair in model:
                model[pair] += 1
            # Nếu key chưa tồn tại trong model thì bổ sung key vào model với value =1
            else:
                model[pair] = 1
    return model

In [4]:
#Kiểm tra kết quả
freqs = count_freq_words(train_x, train_y)
freqs

{('just', 0): 1,
 ('plain', 0): 1,
 ('boring', 0): 1,
 ('entirely', 0): 1,
 ('predictable', 0): 1,
 ('and', 0): 2,
 ('lacks', 0): 1,
 ('energy', 0): 1,
 ('no', 0): 1,
 ('surprises', 0): 1,
 ('very', 0): 1,
 ('few', 0): 1,
 ('laughs', 0): 1,
 ('very', 1): 1,
 ('powerful', 1): 1,
 ('the', 1): 2,
 ('most', 1): 1,
 ('fun', 1): 1,
 ('film', 1): 1,
 ('of', 1): 1,
 ('summer', 1): 1}

In [5]:
# Hàm lấy ra tần suất xuất hiện là value trong `freq` dựa vào key (word, label)
def lookup(freqs, word, label):
    '''
    Args:
        freqs: một từ điển với tần số của mỗi cặp
        word: từ để tra cứu
        label: nhãn tương ứng với từ
    Output:
        count: số lần từ có nhãn tương ứng xuất hiện.
    '''
    count = 0

    pair = (word, label)
    if pair in freqs:
        count = freqs[pair]

    return count

lookup(freqs, "just", 0)

1

**Thuật toán phân loại Naive Bayes**

**Bước 1: Tính xác suất tiên nghiệm của các class**

In [6]:
def compute_prior_prob(train_y):
    # Tính D, D_pos, D_neg dựa vào x_train
    # Tính D, số lượng các sample trong training
    D = len(train_y)

    # Tính D_pos, số lượng các positive sample trong training
    D_pos = len(list(filter(lambda x: x == 1, train_y)))

    # Tính D_neg, số lượng các negative sample trong training
    D_neg = len(list(filter(lambda x: x == 0, train_y)))

    # Tính xác suất tiên nghiệm cho các class 0 và 1
    p_prior = {0:(D_neg/D), 1:(D_pos/D)}
    return p_prior

In [7]:
#Kiểm tra kết quả
compute_prior_prob(train_y)

{0: 0.6, 1: 0.4}

**Bước 2: Tính xác suất likelihood**

In [8]:
def compute_likelihood(freqs):
    # Tính xác suất likelihood của mỗi từ trong bộ từ điển

    # Tính V các từ duy nhất xuất hiện trong tập train
    vocab = set([pair[0] for pair in freqs.keys()])
    V = len(vocab)

    # Tính N_pos: số lượng từ trong positive samples và N_neg: số từ trong negative sample
    N_pos = N_neg = 0
    for pair in freqs.keys():
        # Nếu như class: 1 tăng N_pos thêm số lần xuất hiện của pair trong freqs
        if pair[1] > 0:
            N_pos += freqs[pair]

        # Nếu như class: 0 tăng N_neg thêm số lần xuất hiện của pair trong freqs
        else:
            N_neg += freqs[pair]
    
    print(f'V: {V}, N_pos: {N_pos}, N_neg: {N_neg}')

    # Tính likelihood cho mỗi từ trong bộ từ điển
    p_likelihood = {}
    for word in vocab:
        # Lấy tần xuất xuất hiện của mỗi từ là positive hoặc negative
        freq_pos = lookup(freqs, word, 1)
        freq_neg = lookup(freqs, word, 0)

        # Tính xác suất likelihood của mỗi từ với class positive và negative
        p_w_pos = (freq_pos + 1) / (N_pos + V)
        p_w_neg = (freq_neg + 1) / (N_neg + V)

        # Lưu vào p_likelihood dictionary
        p_likelihood[word] = {0:p_w_neg, 1:p_w_pos}
    
    return p_likelihood

In [9]:
#Kiểm tra kết quả
compute_likelihood(freqs)

V: 20, N_pos: 9, N_neg: 14


{'few': {0: 0.058823529411764705, 1: 0.034482758620689655},
 'surprises': {0: 0.058823529411764705, 1: 0.034482758620689655},
 'lacks': {0: 0.058823529411764705, 1: 0.034482758620689655},
 'summer': {0: 0.029411764705882353, 1: 0.06896551724137931},
 'powerful': {0: 0.029411764705882353, 1: 0.06896551724137931},
 'entirely': {0: 0.058823529411764705, 1: 0.034482758620689655},
 'very': {0: 0.058823529411764705, 1: 0.06896551724137931},
 'no': {0: 0.058823529411764705, 1: 0.034482758620689655},
 'laughs': {0: 0.058823529411764705, 1: 0.034482758620689655},
 'of': {0: 0.029411764705882353, 1: 0.06896551724137931},
 'the': {0: 0.029411764705882353, 1: 0.10344827586206896},
 'fun': {0: 0.029411764705882353, 1: 0.06896551724137931},
 'most': {0: 0.029411764705882353, 1: 0.06896551724137931},
 'plain': {0: 0.058823529411764705, 1: 0.034482758620689655},
 'just': {0: 0.058823529411764705, 1: 0.034482758620689655},
 'predictable': {0: 0.058823529411764705, 1: 0.034482758620689655},
 'film': {0:

**Bước 3: Hoàn thiện train function cho Naive Bayes**

In [10]:
def train_naive_bayes(train_x, train_y):
    ''' Huấn luyện thuật toán Naive Bayes
    Args:
        train_x: Danh sách các câu
        train_y: Danh sách các nhãn tương ứng (0 hoặc 1)
    Output:
        p_prior: Xác suất tiên nghiệm
        p_likelihood: giá trị likelihood tối đa của xác suất.
    '''
    # Xây dựng từ điển tần suất xuất hiện của từ và nhãn tương ứng
    freqs = count_freq_words(train_x, train_y)

    # Tính xác suất tiên nghiệm
    p_prior = compute_prior_prob(train_y)

    # Tính xác suất likelihood
    p_likelihood = compute_likelihood(freqs)

    return p_prior, p_likelihood

In [11]:
# Kiểm tra kết quả
p_prior, p_likelihood = train_naive_bayes(train_x, train_y)
p_prior, p_likelihood

V: 20, N_pos: 9, N_neg: 14


({0: 0.6, 1: 0.4},
 {'few': {0: 0.058823529411764705, 1: 0.034482758620689655},
  'surprises': {0: 0.058823529411764705, 1: 0.034482758620689655},
  'lacks': {0: 0.058823529411764705, 1: 0.034482758620689655},
  'summer': {0: 0.029411764705882353, 1: 0.06896551724137931},
  'powerful': {0: 0.029411764705882353, 1: 0.06896551724137931},
  'entirely': {0: 0.058823529411764705, 1: 0.034482758620689655},
  'very': {0: 0.058823529411764705, 1: 0.06896551724137931},
  'no': {0: 0.058823529411764705, 1: 0.034482758620689655},
  'laughs': {0: 0.058823529411764705, 1: 0.034482758620689655},
  'of': {0: 0.029411764705882353, 1: 0.06896551724137931},
  'the': {0: 0.029411764705882353, 1: 0.10344827586206896},
  'fun': {0: 0.029411764705882353, 1: 0.06896551724137931},
  'most': {0: 0.029411764705882353, 1: 0.06896551724137931},
  'plain': {0: 0.058823529411764705, 1: 0.034482758620689655},
  'just': {0: 0.058823529411764705, 1: 0.034482758620689655},
  'predictable': {0: 0.058823529411764705, 1: 

**Dự đoán với các mẫu thử nghiệm**

In [12]:
def naive_bayes_predict(sentence, p_prior, p_likelihood):
    '''
    Args:
        sentence: một câu
        p_prior: một từ điển của xác suất tiên nghiệm
        p_likelihood: một từ điển các từ ánh xạ tới xác suất
    Output:
        p: xác suất của chuỗi đó gắn với 0: negative, 1: positive

    '''
    # Tiền xử lý dữ liệu
    words = basic_preprocess(sentence)

    # Khởi tạo giá trị xác suất ban đầu là giá trị xác suất tiên nghiệm
    p_neg = p_prior[0]
    p_pos = p_prior[1]

    for word in words:
        # Kiểm tra xem word có tồn tại trong p_likelihood hay không
        if word in p_likelihood:
            # nhân xác suất tiên nghiệm với xác suất likelihood của các từ
            p_neg *= p_likelihood[word][0]
            p_pos *= p_likelihood[word][1]
    
    return {'prob': {0: p_neg, 1: p_pos},
            'label': 0 if p_neg > p_pos else 1}

In [13]:
# Kiểm tra kết quả
sentence = "predictable with no fun"
naive_bayes_predict(sentence, p_prior, p_likelihood)

{'prob': {0: 6.106248727864848e-05, 1: 3.2801672885317154e-05}, 'label': 0}

# Naive Bayes Classfier for Sentiment Analysis on Tweets
**Phân tích cảm xúc trên tập 1Tweets1 sử dụng thuật toán phân loại Naive Bayes**

In [14]:
import re
import string
import nltk
import numpy as np
nltk.download('twitter_samples')
from nltk.corpus import twitter_samples
from nltk.tokenize import TweetTokenizer
from tqdm import tqdm

[nltk_data] Downloading package twitter_samples to /root/nltk_data...
[nltk_data]   Package twitter_samples is already up-to-date!


**Download Dataset**

In [15]:
# Tải về tập dữ liệu tweets
all_positive_tweets = twitter_samples.strings('positive_tweets.json')
all_negative_tweets = twitter_samples.strings('negative_tweets.json')

# Chia thành 2 tập train và test
# train: 4000 samples, test: 1000 samples
train_pos = all_positive_tweets[:4000]
test_pos = all_positive_tweets[4000:]

train_neg = all_negative_tweets[:4000]
test_neg = all_negative_tweets[4000:]

train_x = train_pos + train_neg
test_x = test_pos + test_neg

# Tạo nhãn negative: 0, positive: 1
train_y = np.append(np.ones(len(train_pos)), np.zeros(len(train_neg)))
test_y = np.append(np.ones(len(test_pos)), np.zeros(len(test_neg)))

**Tiền xử lý dữ liệu cho tập Tweets**

In [16]:
all_positive_tweets[:10]

['#FollowFriday @France_Inte @PKuchly57 @Milipol_Paris for being top engaged members in my community this week :)',
 '@Lamb2ja Hey James! How odd :/ Please call our Contact Centre on 02392441234 and we will be able to assist you :) Many thanks!',
 '@DespiteOfficial we had a listen last night :) As You Bleed is an amazing track. When are you in Scotland?!',
 '@97sides CONGRATS :)',
 'yeaaaah yippppy!!!  my accnt verified rqst has succeed got a blue tick mark on my fb profile :) in 15 days',
 '@BhaktisBanter @PallaviRuhail This one is irresistible :)\n#FlipkartFashionFriday http://t.co/EbZ0L2VENM',
 "We don't like to keep our lovely customers waiting for long! We hope you enjoy! Happy Friday! - LWWF :) https://t.co/smyYriipxI",
 '@Impatientraider On second thought, there’s just not enough time for a DD :) But new shorts entering system. Sheep must be buying.',
 'Jgh , but we have to go to Bayan :D bye',
 'As an act of mischievousness, am calling the ETL layer of our in-house warehousing 

In [17]:
def basic_preprocess(text):
    '''
    Args:
        text: câu đầu vào
    Output:
        text_clean: danh sách các từ (token) sau khi chuyển sang chữ thường và
            được phân tách bởi khoảng trắng
    '''
    # xóa bỏ stock market tickers like $GE
    text = re.sub(r'\$\w*', '', text)

    # xóa bỏ old style retweet text "RT"
    text = re.sub(r'^RT[\s]+', '', text)

    # xóa bỏ hyperlinks
    text = re.sub(r'https?:\/\/.*[\r\n]*', '', text)

    # xóa bỏ hashtags
    text = re.sub(r'#', '', text)

    # tokenize
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True, reduce_len=True)
    text_tokens = tokenizer.tokenize(text)

    text_clean = []
    for word in text_tokens:
        if word not in string.punctuation:  # remove punctuation
            text_clean.append(word)

    return text_clean

In [18]:
#Kiểm tra kết quả
example_sentence = "RT @Twitter @chapagain Hello There! Have a great day. #good #morning http://chapagain.com.np"
basic_preprocess(example_sentence)

['hello', 'there', 'have', 'a', 'great', 'day', 'good', 'morning']

**Huấn luyện Naive Bayes Classifier trên tập Tweets**

In [19]:
p_prior, p_likelihood = train_naive_bayes(train_x, train_y)

V: 10841, N_pos: 41916, N_neg: 43321


In [20]:
#Kết quả ví dụ về xác suất tiên nghiệm và likelihood của từ happy
p_prior, p_likelihood['happy']

({0: 0.5, 1: 0.5}, {0: 0.0003507994534913777, 1: 0.0028432245957882366})

**Dự đoán**

In [21]:
test_x[0], test_y[0]

('Bro:U wan cut hair anot,ur hair long Liao bo\nMe:since ord liao,take it easy lor treat as save $ leave it longer :)\nBro:LOL Sibei xialan',
 1.0)

In [22]:
naive_bayes_predict(test_x[0], p_prior, p_likelihood)

{'prob': {0: 1.1032955242760357e-80, 1: 2.0993946427583525e-79}, 'label': 1}

**Đánh giá độ chính xác trên tập test**

In [23]:
acc = 0
for sentence, label in zip(test_x, test_y):

    # dự đoán từng câu trong tập kiểm tra
    pred = naive_bayes_predict(sentence, p_prior, p_likelihood)['label']

    # so sánh nhãn dự đoán với nhãn thực tế
    if int(pred) == int(label):
        acc += 1

print('Accuracy: ', acc/len(test_x))

Accuracy:  0.994
